In [29]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from numpy.random import randn
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
data = pd.read_csv('dataset/diabetes_temporary.csv')
print (data.shape)
print (data.tail())
print (data.columns)

(768, 9)
     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  Outcome  
763                     0.171   63        0  
764                     0.340   27        0  
765                     0.245   30        0  
766                     0.349   47        1  
767                     0.315   23        0  
Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')


In [30]:
features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
label = ['Outcome']
X = data[features]
y = data[label] 

In [31]:
X_true_train, X_true_test, y_true_train, y_true_test = train_test_split(X, y, test_size=0.30, random_state=42)
clf_true = RandomForestClassifier(n_estimators=100)
clf_true.fit(X_true_train,y_true_train)
y_true_pred=clf_true.predict(X_true_test)
print("Base Accuracy:",metrics.accuracy_score(y_true_test, y_true_pred))
print("Base classification report:",metrics.classification_report(y_true_test, y_true_pred))

c:\Users\ensar\Desktop\Diabetes-Prediction\venv\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Base Accuracy: 0.7705627705627706
Base classification report:               precision    recall  f1-score   support

           0       0.82      0.83      0.83       151
           1       0.68      0.65      0.66        80

    accuracy                           0.77       231
   macro avg       0.75      0.74      0.74       231
weighted avg       0.77      0.77      0.77       231



In [32]:
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [33]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = generator.predict(x_input)
    y = np.zeros((n_samples, 1))
    
    return X, y

In [34]:
# generate n real samples with class labels; We randomly select n samples from the real data
def generate_real_samples(n):
    X = data.sample(n)
    y = np.ones((n, 1))
    return X, y

In [35]:
def define_generator(latent_dim, n_outputs=9):
    model = Sequential()
    model.add(Dense(15, activation='relu',  kernel_initializer='he_uniform', input_dim=latent_dim))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(n_outputs, activation='linear'))
    return model

In [36]:
generator1 = define_generator(10, 9)
generator1.summary()

c:\Users\ensar\Desktop\Diabetes-Prediction\venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ (None, 15)             │           165 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 30)             │           480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 9)              │           279 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 924 (3.61 KB)

 Trainable params: 924 (3.61 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
def define_discriminator(n_inputs=9):
    model = Sequential()
    model.add(Dense(25, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [38]:
discriminator1 = define_discriminator(9)
discriminator1.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_27 (Dense)                │ (None, 25)             │           250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 50)             │         1,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,601 (6.25 KB)

 Trainable params: 1,601 (6.25 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

In [40]:
# create a line plot of loss for the gan and save to file
def plot_history(d_hist, g_hist):
    # plot loss
    plt.subplot(1, 1, 1)
    plt.plot(d_hist, label='d')
    plt.plot(g_hist, label='gen')
    plt.show()
    plt.close()

In [50]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=3000, n_batch=128, n_eval=200):
    # determine half the size of one batch, for updating the  discriminator
    half_batch = int(n_batch / 2)
    d_history = []
    g_history = []
    # manually enumerate epochs
    for epoch in range(n_epochs):
    
      # prepare real samples
      x_real, y_real = generate_real_samples(half_batch)
      # prepare fake examples
      x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      # update discriminator
      d_loss_real, d_real_acc = d_model.train_on_batch(x_real, y_real)
      d_loss_fake, d_fake_acc = d_model.train_on_batch(x_fake, y_fake)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
      # prepare points in latent space as input for the generator
      x_gan = generate_latent_points(latent_dim, n_batch)
      # create inverted labels for the fake samples
      y_gan = np.ones((n_batch, 1))
      # update the generator via the discriminator's error
      g_loss_fake = gan_model.train_on_batch(x_gan, y_gan)
      print('EPOCH: ' + str(epoch+1))
      d_history.append(d_loss)
      g_history.append(g_loss_fake)
    plot_history(d_history, g_history)
    g_model.save('trained_generated_model2.h5')

In [51]:
# size of the latent space
latent_dim = 10
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim)

c:\Users\ensar\Desktop\Diabetes-Prediction\venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


c:\Users\ensar\Desktop\Diabetes-Prediction\venv\lib\site-packages\keras\src\backend\tensorflow\trainer.py:71: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


EPOCH: 1
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
EPOCH: 2
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 3
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 4
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 6
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
EPOCH: 7
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
EPOCH: 8
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
EPOCH: 9
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 11
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 12
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 13
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 14
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
EPOCH: 15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
EPOCH: 16
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
EPOCH: 17
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
EPOCH: 18
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
EPOCH: 19
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
EPOCH: 21
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
E

In [44]:
from keras.models import load_model
model =load_model('trained_generated_model.h5')

latent_points = generate_latent_points(10, 750)
X = model.predict(latent_points)
data_fake = pd.DataFrame(data=X,  columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'])
data_fake.head()

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,39.783676,14.734287,5.521362,1.762373,26.084600,28.219048,-8.290518,-29.035463,29.534822
1,26.852383,8.825723,3.678695,2.022432,17.127092,18.450165,-5.503119,-18.635731,18.782093
2,35.925632,13.087619,4.819084,2.181347,22.825949,27.344566,-7.712211,-26.482714,26.948925
3,19.687561,6.549464,2.842821,0.892720,12.243945,14.657292,-4.641095,-15.249620,14.194225
4,23.695631,7.995221,3.649239,1.326724,15.694657,16.468304,-5.675329,-18.695581,16.197935


In [45]:
outcome_mean = data_fake.Outcome.mean()
data_fake['Outcome'] = data_fake['Outcome'] > outcome_mean
data_fake["Outcome"] = data_fake["Outcome"].astype(int)

In [46]:
features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
label = ['Outcome']
X_fake_created = data_fake[features]
y_fake_created = data_fake[label]

In [49]:
X_fake_created.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,39.783676,14.734287,5.521362,1.762373,26.084600,28.219048,-8.290518,-29.035463
1,26.852383,8.825723,3.678695,2.022432,17.127092,18.450165,-5.503119,-18.635731
2,35.925632,13.087619,4.819084,2.181347,22.825949,27.344566,-7.712211,-26.482714
3,19.687561,6.549464,2.842821,0.892720,12.243945,14.657292,-4.641095,-15.249620
4,23.695631,7.995221,3.649239,1.326724,15.694657,16.468304,-5.675329,-18.695581
